In [1]:
import pandas as pd
import numpy as np
import glob, os, functools
from ete3 import NCBITaxa

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import seaborn as sns
import pickle, gzip

In [2]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="taxid .* was translated into .*",
    category=UserWarning,
)

In [3]:
%load_ext autoreload
%autoreload 2
import sys
import importlib
sys.path.append("./")

import run_modelling_cleaned as rm
importlib.reload(rm)

<module 'run_modelling_cleaned' from '/home/mercedes/Documents/projects/github/cfHIV/final_repo/cfHIV/notebooks/run_modelling_cleaned.py'>

In [4]:
# To run edgeR
import r_models as R

In [5]:

DOMAINS = ["bacteria", "archaea", "viruses"]
RANKS = ["species", "genus", "family", "order", "class", "phylum"]


def load_domain_results_pickle(path="../tables/domain_results.pkl.gz"):
    with gzip.open(path, "rb") as f:
        return pickle.load(f)

domain_results = load_domain_results_pickle("../tables/domain_results.pkl.gz")

In [6]:
DOMAIN = "bacteria"
rank = "genus"

counts = domain_results[DOMAIN]["rank_tables"][rank]
meta = domain_results[DOMAIN]["meta"]

In [7]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

ro.r("library(edgeR)")

ro.r("""
read_counts_df <- function(x) {
  y <- DGEList(counts=x, genes=rownames(x))
  keep <- rowSums(cpm(y) > 5) >= 10
  y <- y[keep,, keep.lib.sizes=FALSE]
  y <- calcNormFactors(y)
  return(y)
}

run_de <- function(y, design) {
  y <- estimateDisp(y, design)
  fit <- glmQLFit(y, design)
  return(fit)
}
""")

R callback write-console: Loading required package: limma
  


In [ ]:
import os, pickle, gzip
import pandas as pd

def save_gzip_pickle(obj, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with gzip.open(path, "wb") as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)

# --- plasma vs water ---
res_bg, logcpm_bg, meta_pw = R.edger_plasma_vs_water(counts, meta)

save_gzip_pickle(res_bg,   "../tables/edger_plasma_vs_water_res.pkl.gz")
save_gzip_pickle(logcpm_bg,"../tables/edger_plasma_vs_water_logcpm.pkl.gz")
save_gzip_pickle(meta_pw,  "../tables/edger_plasma_vs_water_meta.pkl.gz")


In [8]:
design_df = pd.read_table("../tables/edgeR_design.txt", index_col=0)


In [9]:


def save_gzip_pickle(obj, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with gzip.open(path, "wb") as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)


# --- bnabs in plasma ---
res_bn, logcpm_bn, design_aligned = R.edger_bnabs_plasma(counts, meta, design_df, bNAbs_ref=None)

save_gzip_pickle(res_bn,          "../tables/edger_bnabs_plasma_res.pkl.gz")
save_gzip_pickle(logcpm_bn,       "../tables/edger_bnabs_plasma_logcpm.pkl.gz")
save_gzip_pickle(design_aligned,  "../tables/edger_bnabs_plasma_design.pkl.gz")